# Chapitre 2 : Hexapawn

Dans ce chapitre 2, nous mettrons en œuvre notre propre moteur de jeu basé sur un réseau neuronal, similaire à AlphaZero ou Leela Chess Zero. Le jeu d'échecs étant trop complexe pour être entraîné efficacement sur un PC standard, nous nous limiterons à une variante plus simple du jeu d'échecs appelée Hexapawn. Cependant, comme toutes les méthodes existent et que l'approche d'AlphaZero est assez indifférente aux règles particulières d'un jeu, cette mise en œuvre peut facilement être étendue à des problèmes plus complexes, y compris les échecs - à condition que les énormes ressources informatiques nécessaires soient disponibles.

## 1. Générer des données d'entraînement

On va générer des données d'entraînement pour l'apprentissage supervisé en résolvant le jeu avec minimax.
On a commencer par réimplémenter l'algorithme Minimax pour évaluer les configurations du jeu. Ensuite, on a créer une class python Board qui permet de représenter l'état du jeu et de générer les données d'entraînement.

In [2]:
from common.game import Board  # Importation de la classe Board depuis le module common.game
from common.mnx_minimax import minimax  # Importation de la fonction minimax depuis le module mnx_minimax
import copy  # Importation de la bibliothèque copy pour créer des copies d'objets
import numpy as np  # Importation de numpy pour les opérations sur les tableaux

# Définition d'une fonction pour obtenir le meilleur coup et sa valeur pour un plateau donné
def getBestMoveRes(board):
    bestMove = None
    bestVal = 1000000000  # Initialisation à une valeur élevée
    if(board.turn == board.WHITE):
        bestVal = -1000000000  # Si c'est le tour des blancs, initialise à une valeur très basse
    for m in board.generateMoves():  # Parcourt tous les mouvements possibles
        tmp = copy.deepcopy(board)  # Crée une copie du plateau pour simuler le mouvement
        tmp.applyMove(m)  # Applique le mouvement sur la copie
        mVal = minimax(tmp, 30, tmp.turn == board.WHITE)  # Calcule la valeur du mouvement avec Minimax
        if(board.turn == board.WHITE and mVal > bestVal):
            bestVal = mVal
            bestMove = m
        if(board.turn == board.BLACK and mVal < bestVal):
            bestVal = mVal
            bestMove = m
    return bestMove, bestVal

# Initialisation des listes pour stocker les données
positions = []
moveProbs = []
outcomes = []
terminals = []  # Liste pour stocker le nombre de terminaux rencontrés

# Fonction récursive pour visiter tous les nœuds de l'arbre de recherche
def visitNodes(board):
    term, _ = board.isTerminal()  # Vérifie si le jeu est terminé
    if(term):
        terminals.append(1)  # Incrémente le compteur de terminaux
        return
    else:
        # Obtient le meilleur coup et sa valeur pour le plateau actuel
        bestMove, bestVal = getBestMoveRes(board)
        positions.append(board.toNetworkInput())  # Ajoute la représentation du plateau pour le réseau
        moveProb = [ 0 for x in range(0,28) ]  # Crée une liste de probabilités de mouvement
        idx = board.getNetworkOutputIndex(bestMove)  # Obtient l'index correspondant au meilleur mouvement
        moveProb[idx] = 1  # Définit la probabilité du meilleur mouvement à 1
        moveProbs.append(moveProb)  # Ajoute les probabilités de mouvement
        # Détermine le résultat du meilleur coup et l'ajoute à la liste des résultats
        if(bestVal > 0):
            outcomes.append(1)
        if(bestVal == 0):
            outcomes.append(0)
        if(bestVal < 0):
            outcomes.append(-1)
        # Génère tous les mouvements possibles pour le prochain état du plateau et répète le processus
        for m in board.generateMoves():
            next = copy.deepcopy(board)
            next.applyMove(m)
            visitNodes(next)

# Crée un plateau et commence la visite des nœuds à partir de la position de départ
board = Board()
board.setStartingPosition()
visitNodes(board)

# Enregistre les données collectées sous forme de fichiers numpy
np.save("positions", np.array(positions))
np.save("moveprobs", np.array(moveProbs))
np.save("outcomes", np.array(outcomes))


On affiche les en-têtes de nos données :

In [3]:
np.load("positions.npy")
np.load("moveprobs.npy")
np.load("outcomes.npy")
print("Positions: ", positions)
print("MoveProbs: ", moveProbs)
print("Outcomes: ", outcomes)

Positions:  [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0,

Ensuite on forme le réseau avec un apprentissage supervisé à partir des données qui a été créé en utilisant la recherche minimax.

In [4]:
# Importation des bibliothèques nécessaires
from keras.models import Model
from keras.layers import *
import numpy as np
import tensorflow as tf

# Définition de l'entrée du modèle avec une forme de (21,)
inp = Input((21,))

# Construction des couches du réseau de neurones
l1 = Dense(128, activation='relu')(inp)
l2 = Dense(128, activation='relu')(l1)
l3 = Dense(128, activation='relu')(l2)
l4 = Dense(128, activation='relu')(l3)
l5 = Dense(128, activation='relu')(l4)

# Définition des têtes de sortie pour la politique et l'évaluation
policyOut = Dense(28, name='policyHead', activation='softmax')(l5)
valueOut = Dense(1, activation='tanh', name='valueHead')(l5)

# Définition de la fonction de perte pour la politique
bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# Création du modèle en spécifiant les entrées et les sorties
model = Model(inp, [policyOut,valueOut])

# Compilation du modèle avec un optimiseur SGD et des fonctions de perte pour chaque tête de sortie
model.compile(optimizer='SGD', loss={'valueHead': 'mean_squared_error', 'policyHead': bce})

# Chargement des données d'entraînement à partir des fichiers positions.npy, moveprobs.npy et outcomes.npy
inputData = np.load("positions.npy")
policyOutcomes = np.load("moveprobs.npy")
valueOutcomes = np.load("outcomes.npy")

# Affichage des formes des données chargées
print(policyOutcomes.shape)
print(inputData.shape)

# Entraînement du modèle avec les données d'entrée et de sortie sur 512 époques et une taille de lot de 16
model.fit(inputData, [policyOutcomes, valueOutcomes], epochs=512, batch_size=16)

# Sauvegarde du modèle entraîné sous forme de fichier .keras
model.save('supervised_model.keras')

(118, 28)
(118, 21)
Epoch 1/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.3510  
Epoch 2/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2760 
Epoch 3/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2115 
Epoch 4/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.1624 
Epoch 5/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.0628 
Epoch 6/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.0420 
Epoch 7/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.9503 
Epoch 8/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.8446 
Epoch 9/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.8971 
Epoch 10/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.8212 
Epoch 11/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6832 
Epoch 12/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7287 
Epoch 13/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6238 
Epoch 14/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5839 
Epoch 15/512
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5472

On teste maitenant le réseau contre un algorithme qui joue aléatoirement.

In [5]:
# Importation des bibliothèques nécessaires
import keras
from common.game import Board  # Importation de la classe Board depuis le fichier common.game
import random  # Importation du module random pour générer des mouvements aléatoires
import numpy as np  # Importation du module numpy pour manipuler des tableaux de données

# Chargement du modèle entraîné à partir du fichier "supervised_model.keras"
model = keras.models.load_model("supervised_model.keras")

# Fonction pour extraire le premier élément d'un tuple
def fst(a):
    return a[0]

# Partie du code pour jouer entre un joueur aléatoire et le réseau de neurones supervisé
def rand_vs_net(board):
    record = []  # Liste pour enregistrer les mouvements effectués
    while(not fst(board.isTerminal())):  # Tant que la partie n'est pas terminée
        if(board.turn == Board.WHITE):  # Si c'est au tour du joueur blanc (aléatoire)
            moves = board.generateMoves()  # Générer les mouvements possibles pour le joueur
            m = moves[random.randint(0, len(moves)-1)]  # Choisir un mouvement aléatoire
            board.applyMove(m)  # Appliquer le mouvement sur le plateau
            record.append(m)  # Enregistrer le mouvement
            continue  # Passer au prochain tour
        else:  # Si c'est au tour du joueur noir (réseau de neurones supervisé)
            q = model.predict(np.array([board.toNetworkInput()]))  # Prédire les probabilités des mouvements
            masked_output = [ 0 for x in range(0,28)]  # Initialiser une liste pour stocker les probabilités masquées
            for m in board.generateMoves():  # Pour chaque mouvement possible
                m_idx = board.getNetworkOutputIndex(m)  # Obtenir l'indice correspondant dans les sorties du réseau
                masked_output[m_idx] = q[0][0][m_idx]  # Mettre à jour la probabilité masquée
            best_idx = np.argmax(masked_output)  # Trouver l'indice du mouvement avec la plus grande probabilité
            sel_move = None  # Initialiser la variable pour stocker le mouvement sélectionné
            for m in board.generateMoves():  # Pour chaque mouvement possible
                m_idx = board.getNetworkOutputIndex(m)  # Obtenir l'indice correspondant dans les sorties du réseau
                if(best_idx == m_idx):  # Si c'est le mouvement avec la plus grande probabilité
                    sel_move = m  # Sélectionner ce mouvement
            board.applyMove(sel_move)  # Appliquer le mouvement sur le plateau
            record.append(sel_move)  # Enregistrer le mouvement
            continue  # Passer au prochain tour
    terminal, winner = board.isTerminal()  # Vérifier si la partie est terminée et le gagnant
    return winner  # Retourner le gagnant de la partie

# Partie du code pour jouer entre deux joueurs aléatoires
def rand_vs_rand(board):
    while(not fst(board.isTerminal())):  # Tant que la partie n'est pas terminée
        moves = board.generateMoves()  # Générer les mouvements possibles pour le joueur
        m = moves[random.randint(0, len(moves)-1)]  # Choisir un mouvement aléatoire
        board.applyMove(m)  # Appliquer le mouvement sur le plateau
        continue  # Passer au prochain tour
    terminal, winner = board.isTerminal()  # Vérifier si la partie est terminée et le gagnant
    return winner  # Retourner le gagnant de la partie

# Initialiser les compteurs de victoires pour les deux joueurs
whiteWins = 0
blackWins = 0

# Jouer 100 parties entre un joueur aléatoire et le réseau de neurones supervisé
for i in range(0,100):
    board = Board()  # Créer un nouveau plateau de jeu
    board.setStartingPosition()  # Placer les pièces sur la position de départ
    moves = board.generateMoves()  # Générer les mouvements possibles pour le joueur
    m = moves[random.randint(0, len(moves)-1)]  # Choisir un mouvement aléatoire
    board.applyMove(m)  # Appliquer le mouvement sur le plateau
    winner = rand_vs_net(board)  # Jouer la partie entre le joueur aléatoire et le réseau de neurones supervisé
    if(winner == Board.WHITE):  # Si le joueur blanc (aléatoire) gagne
        whiteWins += 1  # Incrémenter le nombre de victoires pour le joueur blanc
    if(winner == Board.BLACK):  # Si le joueur noir (réseau de neurones) gagne
        blackWins += 1  # Incrémenter le nombre de victoires pour le joueur noir

all = whiteWins + blackWins  # Total des parties jouées
# Afficher le taux de victoires pour le joueur blanc et le joueur noir
print("Rand vs Supervised Network: "+str(whiteWins/all) + "/"+str(blackWins/all))

# Réinitialiser les compteurs de victoires pour les deux joueurs
whiteWins = 0
blackWins = 0

# Jouer 100 parties entre deux joueurs aléatoires
for i in range(0,100):
    board = Board()  # Créer un nouveau plateau de jeu
    board.setStartingPosition()  # Placer les pièces sur la position de départ
    moves = board.generateMoves()  # Générer les mouvements possibles pour le joueur
    m = moves[random.randint(0, len(moves)-1)]  # Choisir un mouvement aléatoire
    board.applyMove(m)  # Appliquer le mouvement sur le plateau
    winner = rand_vs_rand(board)  # Jouer la partie entre deux joueurs aléatoires
    if(winner == Board.WHITE):  # Si le joueur blanc gagne
        whiteWins += 1  # Incrémenter le nombre de victoires pour le joueur blanc
    if(winner == Board.BLACK):  # Si le joueur noir gagne
        blackWins += 1  # Incrémenter le nombre de victoires pour le joueur noir

all = whiteWins + blackWins  # Total des parties jouées
# Afficher le taux de victoires pour le joueur blanc et le joueur noir
print("Rand vs Rand Network: "+str(whiteWins/all) + "/"+str(blackWins/all))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━